In [1]:
!pip install tensorflow

In [2]:
!pip install scipy

In [3]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
# Caminhos para as pastas de dados
train_data_dir = './data'
batch_size = 32 # ***
image_size = (150, 150)
num_epochs = 50
num_classes = 2

In [5]:
# Crie geradores de dados de treinamento
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, # Normalização das imagens
    validation_split=0.25,  # Define a divisão de validação
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)    

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 41 images belonging to 2 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 12 images belonging to 2 classes.


In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_size[0], image_size[1], 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Camada de saída com 1 unidade para classificação binária

In [9]:
# Compile o modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
# Treine o modelo
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator)

# Salve o modelo treinado
model.save('modeloClassImagens2.h5')

print('Modelo salvo com sucesso!')

Epoch 1/50
2/2 [==============================] - 1s 498ms/step - loss: 2.5062 - accuracy: 0.4146 - val_loss: 0.6419 - val_accuracy: 0.5833
Epoch 2/50
2/2 [==============================] - 0s 177ms/step - loss: 1.2377 - accuracy: 0.5854 - val_loss: 1.2728 - val_accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 1s 386ms/step - loss: 1.1051 - accuracy: 0.5854 - val_loss: 0.6528 - val_accuracy: 0.8333
Epoch 4/50
2/2 [==============================] - 0s 167ms/step - loss: 0.7845 - accuracy: 0.5366 - val_loss: 0.6852 - val_accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 0s 167ms/step - loss: 0.7048 - accuracy: 0.5610 - val_loss: 0.6422 - val_accuracy: 0.5000
Epoch 6/50
2/2 [==============================] - 0s 344ms/step - loss: 0.6761 - accuracy: 0.5366 - val_loss: 0.6581 - val_accuracy: 0.7500
Epoch 7/50
2/2 [==============================] - 0s 170ms/step - loss: 0.6264 - accuracy: 0.7561 - val_loss: 0.6664 - val_accuracy: 0.6667
Epoch 8/50
2/2 [====

c:\Users\pablo\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
